In [ ]:
library(Signac)
library(Seurat)
library(GenomicRanges)
library(future)
library(ggplot2)
library(patchwork)
library(EnsDb.Hsapiens.v86)
library(SingleCellExperiment)
suppressPackageStartupMessages(library(scDblFinder))
library(tidyverse)

In [ ]:
# ----------------set path
setwd("...")


In [ ]:
# ----------------load annotations
annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
# change to UCSC style since the data was mapped to hg38
seqlevels(annotations) <- paste0('chr', seqlevels(annotations))
genome(annotations) <- "hg38"

# Creating a commom peak set

In [ ]:
# read in peak sets
peaks_dmso <- read.table(
  file = ".../DMSO_report/filtered_peak_bc_matrix/peaks.bed",
  col.names = c("chr", "start", "end")
)
peaks_inh21 <- read.table(
  file = ".../Inhibitor_21_report/filtered_peak_bc_matrix/peaks.bed",
  col.names = c("chr", "start", "end")
)

In [ ]:
# convert to genomic ranges
gr_dmso <- makeGRangesFromDataFrame(peaks_dmso)
gr_inh21 <- makeGRangesFromDataFrame(peaks_inh21)
# Create a unified set of peaks to quantify in each dataset
combined_peaks <- GenomicRanges::reduce(c(gr_dmso, gr_inh21))
# Filter out bad peaks based on length
peakwidths <- width(combined_peaks)
# filter
combined_peaks <- combined_peaks[peakwidths  < 10000 & peakwidths > 20]

# Create Fragment objects

In [ ]:
# load metadata
md_dmso <- read.table(
  file = ".../DMSO_report/singlecell.csv",
  stringsAsFactors = FALSE,
  sep = ",",
  header = TRUE,
  row.names = 1
)[-1, ] # remove the first row

md_inh21 <- read.table(
  file = ".../Inhibitor_21_report/singlecell.csv",
  stringsAsFactors = FALSE,
  sep = ",",
  header = TRUE,
  row.names = 1
)[-1, ]
# perform an initial filtering of low count cells
md_dmso <- md_dmso[md_dmso$passed_filters > 500, ]
md_inh21 <- md_inh21[md_inh21$passed_filters > 500, ]
# create fragment objects
frags_dmso <- CreateFragmentObject(
  path = ".../DMSO_report/fragments.tsv.gz",
  cells = rownames(md_dmso)
)
frags_inh21 <- CreateFragmentObject(
  path = ".../Inhibitor_21_report/fragments.tsv.gz",
  cells = rownames(md_inh21)
)

# Quantify peaks in each dataset

In [ ]:
dmso_counts <- FeatureMatrix(
  fragments = frags_dmso,
  features = combined_peaks,
  cells = rownames(md_dmso)
)

inh21_counts <- FeatureMatrix(
  fragments = frags_inh21,
  features = combined_peaks,
  cells = rownames(md_inh21)
)

# Create the objects

In [36]:
dmso_assay <- CreateChromatinAssay(dmso_counts, fragments = frags_dmso,annotation = annotations)
dmso <- CreateSeuratObject(dmso_assay, assay = "ATAC", meta.data=md_dmso)

inh21_assay <- CreateChromatinAssay(inh21_counts, fragments = frags_inh21,annotation = annotations)
inh21 <- CreateSeuratObject(inh21_assay, assay = "ATAC", meta.data=md_inh21)

# Quality control

## dmso

In [ ]:
dmso <- NucleosomeSignal(object = dmso)
dmso <- TSSEnrichment(object = dmso,fast=FALSE)
## add blacklist ratio and fraction of reads in peaks
dmso$pct_reads_in_peaks <- dmso$peak_region_fragments / dmso$passed_filters * 100
dmso$blacklist_ratio <- dmso$blacklist_region_fragments / dmso$peak_region_fragments
dmso$high.tss <- ifelse(dmso$TSS.enrichment > 3, 'High', 'Low')
TSSPlot(dmso, group.by = 'high.tss') + NoLegend()
dmso$nucleosome_group <- ifelse(dmso$nucleosome_signal > 1.2, 'NS > 1.2', 'NS < 1.2')
FragmentHistogram(object = dmso, group.by = 'nucleosome_group')

In [ ]:
#subset once
dmso <- subset(x = dmso,
            subset = nCount_ATAC > 500 &
            nCount_ATAC < 12000 &
            TSS.enrichment > 3 &
            pct_reads_in_peaks > 40 &
            #blacklist_ratio < 0.05 &
            nucleosome_signal < 1.2)

In [ ]:
#subset twice
# https://plger.github.io/scDblFinder/articles/scATAC.html
set.seed(123)
sce <- as.SingleCellExperiment(dmso)
sce <- scDblFinder(sce, aggregateFeatures=TRUE, nfeatures=25, processing="normFeatures")
# identical(colnames(seu), colnames(sce))
dmso[["scDblFinder_score"]] <- sce$scDblFinder.score
dmso[["scDblFinder_class"]] <- sce$scDblFinder.class
dmso <- dmso[,dmso$scDblFinder_class %in% c("singlet")]

## inh21

In [ ]:
inh21 <- NucleosomeSignal(object = inh21)
inh21 <- TSSEnrichment(object = inh21, fast = FALSE)
## add blacklist ratio and fraction of reads in peaks
inh21$pct_reads_in_peaks <- inh21$peak_region_fragments / inh21$passed_filters * 100
inh21$blacklist_ratio <- inh21$blacklist_region_fragments / inh21$peak_region_fragments
inh21$high.tss <- ifelse(inh21$TSS.enrichment > 3, 'High', 'Low')
inh21$nucleosome_group <- ifelse(inh21$nucleosome_signal > 1.2, 'NS > 1.2', 'NS < 1.2')

In [ ]:
inh21 <- subset(x = inh21,
            subset = nCount_ATAC > 500 &
            nCount_ATAC < 12000 &
            TSS.enrichment > 3 &
            pct_reads_in_peaks > 40 &
            blacklist_ratio < 0.05 &
            nucleosome_signal < 1.2)

In [ ]:
sce <- as.SingleCellExperiment(inh21)
sce <- scDblFinder(sce, artificialDoublets=1, aggregateFeatures=TRUE, nfeatures=25, processing="normFeatures")
# identical(colnames(seu), colnames(sce))
inh21[["scDblFinder_score"]] <- sce$scDblFinder.score
inh21[["scDblFinder_class"]] <- sce$scDblFinder.class
inh21 <- inh21[,inh21$scDblFinder_class %in% c("singlet")]


# Merge objects

In [ ]:
dmso <- readRDS(".../dmso_afterfilter.rds")
inh21 <- readRDS(".../inh21_afterfilter.rds")

In [110]:
dmso$treat <- "dmso"
inh21$treat <- "inh21"

In [111]:
combined <- merge(
  x = dmso,
  y = inh21,
  add.cell.ids = c("dmso", "inh21")
)
combined[["ATAC"]]

ChromatinAssay data with 276418 features for 25343 cells
Variable features: 0 
Genome: 
Annotation present: TRUE 
Motifs present: FALSE 
Fragment files: 2 

## Normalization and linear and Non-linear dimension reduction

In [ ]:
# Normalization and linear dimensional reduction
combined <- RunTFIDF(combined)
combined <- FindTopFeatures(combined, min.cutoff = 'q0')
combined <- RunSVD(combined)
options(repr.plot.width=9, repr.plot.height=9)
DepthCor(combined, n =50)

In [ ]:
pc <- 30
# Non-linear dimension reduction
combined <- RunUMAP(combined, dims = 2:pc, reduction = 'lsi')

# Clustering

In [120]:
DefaultAssay(combined) <- "ATAC"

In [121]:
rsl <- seq(0.02, 0.2, by = 0.02)

In [ ]:
combined <- FindNeighbors(combined, reduction = 'lsi', dims = 2:pc) %>% 
                FindClusters(verbose = FALSE,resolution = rsl, algorithm = 3)

In [ ]:
options(repr.plot.width=9, repr.plot.height=9)
DimPlot(object = combined, group.by = 'ATAC_snn_res.0.14',label = TRUE)

# Gene activity together

In [127]:
gene.activities <- GeneActivity(combined)

Extracting gene coordinates

Warning message in SingleFeatureMatrix(fragment = fragments[[x]], features = features, :
"13 features are on seqnames not present in the fragment file. These will be removed."
Extracting reads overlapping genomic regions

Warning message in SingleFeatureMatrix(fragment = fragments[[x]], features = features, :
"13 features are on seqnames not present in the fragment file. These will be removed."
Extracting reads overlapping genomic regions



In [128]:
# add the gene activity matrix to the Seurat object as a new assay and normalize it
combined[['geneactivity']] <- CreateAssayObject(counts = gene.activities)

In [129]:
combined

An object of class Seurat 
296038 features across 25343 samples within 2 assays 
Active assay: ATAC (276418 features, 276418 variable features)
 2 layers present: counts, data
 1 other assay present: geneactivity
 2 dimensional reductions calculated: lsi, umap

In [131]:
DefaultAssay(combined) <- "geneactivity"

In [132]:
combined

An object of class Seurat 
296038 features across 25343 samples within 2 assays 
Active assay: geneactivity (19620 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: ATAC
 2 dimensional reductions calculated: lsi, umap

In [133]:
combined <- FindVariableFeatures(combined)
combined <- NormalizeData(
    object = combined,
    assay = 'geneactivity',
    normalization.method = 'LogNormalize',
    scale.factor = median(combined$nCount_geneactivity)
)
combined <- ScaleData(combined)

Centering and scaling data matrix

